In [1]:
import sys
sys.path.append('/home/s94zalek/shape_matching')

# set logging level to info
import logging
logging.basicConfig(level=logging.INFO)

from datasets_code import build_dataloader, build_dataset
from utils.options import parse_options
from train import create_train_val_dataloader
import torch
import numpy as np

import os
os.chdir('/home/s94zalek/shape_matching')

# print current working directory
print(os.getcwd())

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/home/s94zalek/shape_matching


In [2]:
import os

# root_path = osp.abspath(osp.join(__file__, osp.pardir))
root_path = '/home/s94zalek/shape_matching'

opt = parse_options(root_path, is_train=False, use_argparse=False,
                    opt_path = 'options/train/faust.yaml')

opt['root_path'] = root_path
opt['dist'] = False

opt['datasets']['train_dataset']['return_corr'] = True
opt['datasets']['train_dataset']['return_dist'] = False
opt['datasets']['test_dataset']['return_dist'] = False

# create train and validation dataloaders
result = create_train_val_dataloader(opt)
train_loader, train_sampler, val_loader, total_epochs, total_iters = result

test_set = build_dataset(opt['datasets']['test_dataset'])
test_loader = build_dataloader(
test_set, opt['datasets']['test_dataset'], phase='val', num_gpu=opt['num_gpu'], dist=opt['dist'], sampler=None, seed=opt['manual_seed'])


Backend DataParallel.
Path already exists. Rename it to /home/s94zalek/shape_matching/results/faust_archived_20240503_085119


Calculating functional maps: 100%|████████████████████████████████████████████████████████████████████████████| 6400/6400 [01:19<00:00, 80.10it/s]
2024-05-03 08:52:51,244 INFO: Dataset [PairFaustDataset]-[FaustTrain] is built.
Calculating functional maps: 100%|█████████████████████████████████████████████████████████████████████████████| 400/400 [00:03<00:00, 104.02it/s]
2024-05-03 08:52:57,288 INFO: Dataset [PairFaustDataset]-[FaustTest] is built.
Calculating functional maps: 100%|█████████████████████████████████████████████████████████████████████████████| 400/400 [00:03<00:00, 104.23it/s]
2024-05-03 08:53:01,814 INFO: Dataset [PairFaustDataset]-[FaustTest] is built.


In [3]:
Cxy_40 = []
Dx_40 = []
Dy_40 = []

Vxy_computed_40 = []
Rxy_computed_40 = []

data_40 = []

train_dataset = train_loader.dataset
for i in range(len(train_dataset)):
    data = train_dataset[i]
    if data['first']['name'] == 'tr_reg_040':
        
        Cxy_40.append(data['Cxy'])
        
        Dx_40.append(data['first']['evals'])
        Dy_40.append(data['second']['evals'])
        
        Vxy_computed_40.append(data['Vxy'])
        Rxy_computed_40.append(data['Rxy'])
        
        data_40.append(data)
        
Cxy_40_full = torch.stack(Cxy_40)
Cxy_40_truncated = torch.stack(Cxy_40)[:, :20, :20]

Dx_40_full = torch.stack(Dx_40)
Dy_40_full = torch.stack(Dy_40)

Vxy_computed_40_full = torch.stack(Vxy_computed_40)
Rxy_computed_40_full = torch.stack(Rxy_computed_40)

In [4]:
########################################################################################
# Check if difference operators in dataset are correct
########################################################################################

In [5]:
Cxy_T_Cxy_40_full = torch.bmm(Cxy_40_full.transpose(1, 2), Cxy_40_full)

Rxy_40_left = torch.bmm(
    torch.diag_embed(-1 / Dx_40_full),
    Cxy_40_full.transpose(1, 2)
    )
Rxy_40_right = torch.bmm(
    torch.diag_embed(-Dy_40_full),
    Cxy_40_full
    )

Rxy_40_full = torch.bmm(Rxy_40_left, Rxy_40_right)

In [6]:
print('Vxy in dataset - Vxy computed:', (Vxy_computed_40_full - Cxy_T_Cxy_40_full).sum())
print('Rxy in dataset - Rxy computed:', (Rxy_40_full - Rxy_computed_40_full).sum())

Vxy in dataset - Vxy computed: tensor(0.0002)
Rxy in dataset - Rxy computed: tensor(-6.4227e-05)


In [7]:
########################################################################################
#
# Descriptor preservation by GT Fmap
#
########################################################################################

In [8]:
data = data_40[21]
data_x = data['first']
data_y = data['second']

data['Cxy'].shape, data_x['evecs_trans'].shape, data_y['evecs'].shape

(torch.Size([200, 200]), torch.Size([200, 5001]), torch.Size([5001, 200]))

In [9]:
# indicator functions as descriptors

ind_pres_loss = (data['Cxy'] @ data_x['evecs_trans'] - data_y['evecs_trans']).abs()

print('Indicator preserv. loss,  50 ef, sum:',
      ind_pres_loss[:50, :50].sum(), 'mean:', ind_pres_loss[:50, :50].mean())
print('Indicator preserv. loss, 100 ef, sum:', ind_pres_loss[:100, :100].sum(),
      'mean:', ind_pres_loss[:100, :100].mean())
print('Indicator preserv. loss, 150 ef, sum:', ind_pres_loss[:150, :150].sum(),
      'mean:', ind_pres_loss[:150, :150].mean())
print('Indicator preserv. loss, 200 ef, sum:', ind_pres_loss[:200, :200].sum(),
      'mean:', ind_pres_loss[:200, :200].mean())

Indicator preserv. loss,  50 ef, sum: tensor(0.3631) mean: tensor(0.0001)
Indicator preserv. loss, 100 ef, sum: tensor(1.6721) mean: tensor(0.0002)
Indicator preserv. loss, 150 ef, sum: tensor(3.8614) mean: tensor(0.0002)
Indicator preserv. loss, 200 ef, sum: tensor(5.6738) mean: tensor(0.0001)


In [10]:
# I guess this counts as preservation?

In [11]:
import utils.geometry_util as geometry_util

# hks as descriptors
hks_x = geometry_util.compute_hks_autoscale(data_x['evals'].unsqueeze(0), data_x['evecs'].unsqueeze(0), 16)[0]
hks_y = geometry_util.compute_hks_autoscale(data_y['evals'].unsqueeze(0), data_y['evecs'].unsqueeze(0), 16)[0]

hks_pres_loss = (data['Cxy'] @ data_x['evecs_trans'] @ hks_x - data_y['evecs_trans'] @ hks_y).abs()

print('HKS preserv. loss,  50 ef, sum:', hks_pres_loss[:50, :50].sum(), 'mean:', hks_pres_loss[:50, :50].mean())
print('HKS preserv. loss, 100 ef, sum:', hks_pres_loss[:100, :100].sum(), 'mean:', hks_pres_loss[:100, :100].mean())
print('HKS preserv. loss, 150 ef, sum:', hks_pres_loss[:150, :150].sum(), 'mean:', hks_pres_loss[:150, :150].mean())
print('HKS preserv. loss, 200 ef, sum:', hks_pres_loss[:200, :200].sum(), 'mean:', hks_pres_loss[:200, :200].mean())

HKS preserv. loss,  50 ef, sum: tensor(17.4320) mean: tensor(0.0218)
HKS preserv. loss, 100 ef, sum: tensor(21.4995) mean: tensor(0.0134)
HKS preserv. loss, 150 ef, sum: tensor(24.1054) mean: tensor(0.0100)
HKS preserv. loss, 200 ef, sum: tensor(25.4915) mean: tensor(0.0080)


In [12]:
# wks as descriptors

wks_x = geometry_util.compute_wks_autoscale(data_x['evals'].unsqueeze(0), data_x['evecs'].unsqueeze(0), data_x['mass'].unsqueeze(0))[0]
wks_y = geometry_util.compute_wks_autoscale(data_y['evals'].unsqueeze(0), data_y['evecs'].unsqueeze(0), data_y['mass'].unsqueeze(0))[0]

wks_pres_loss = (data['Cxy'] @ data_x['evecs_trans'] @ wks_x - data_y['evecs_trans'] @ wks_y).abs()

print('WKS preserv. loss,  50 ef, sum:', wks_pres_loss[:50, :50].sum(), 'mean:', wks_pres_loss[:50, :50].mean())
print('WKS preserv. loss, 100 ef, sum:', wks_pres_loss[:100, :100].sum(), 'mean:', wks_pres_loss[:100, :100].mean())
print('WKS preserv. loss, 150 ef, sum:', wks_pres_loss[:150, :150].sum(), 'mean:', wks_pres_loss[:150, :150].mean())
print('WKS preserv. loss, 200 ef, sum:', wks_pres_loss[:200, :200].sum(), 'mean:', wks_pres_loss[:200, :200].mean())

WKS preserv. loss,  50 ef, sum: tensor(25.3663) mean: tensor(0.0101)
WKS preserv. loss, 100 ef, sum: tensor(77.3188) mean: tensor(0.0077)
WKS preserv. loss, 150 ef, sum: tensor(126.9926) mean: tensor(0.0066)
WKS preserv. loss, 200 ef, sum: tensor(149.6696) mean: tensor(0.0058)


In [13]:
########################################################################################
#
# Does this count as descriptor preservation?
# I guess it does
#
########################################################################################

In [14]:
import torch

Cxy_lstsq = torch.linalg.lstsq(
    (data_x['evecs_trans']).T,
    (data_y['evecs_trans']).T
).solution.T

Cxy_hks_lstsq = torch.linalg.lstsq(
    hks_x.T @ data_x['evecs_trans'].T,
    hks_y.T @ data_y['evecs_trans'].T
).solution.T

Cxy_wks_lstsq = torch.linalg.lstsq(
    wks_x.T @ data_x['evecs_trans'].T,
    wks_y.T @ data_y['evecs_trans'].T
).solution.T

print('Cxy_lstsq error,      50 ef, sum:', (Cxy_lstsq - data['Cxy']).abs()[:50, :50].sum(), 'mean:', (Cxy_lstsq - data['Cxy']).abs()[:50, :50].mean())
print('Cxy_lstsq error,     100 ef, sum:', (Cxy_lstsq - data['Cxy']).abs()[:100, :100].sum(), 'mean:', (Cxy_lstsq - data['Cxy']).abs()[:100, :100].mean())
print('Cxy_lstsq error,     200 ef, sum:', (Cxy_lstsq - data['Cxy']).abs().sum(), 'mean:', (Cxy_lstsq - data['Cxy']).abs().mean())
print()
print('Cxy_hks_lstsq error,  50 ef, sum:', (Cxy_hks_lstsq - data['Cxy']).abs()[:50, :50].sum(), 'mean:', (Cxy_hks_lstsq - data['Cxy']).abs()[:50, :50].mean())
print('Cxy_hks_lstsq error, 100 ef, sum:', (Cxy_hks_lstsq - data['Cxy']).abs()[:100, :100].sum(), 'mean:', (Cxy_hks_lstsq - data['Cxy']).abs()[:100, :100].mean())
print('Cxy_hks_lstsq error, 200 ef, sum:', (Cxy_hks_lstsq - data['Cxy']).abs().sum(), 'mean:', (Cxy_hks_lstsq - data['Cxy']).abs().mean())
print()
print('Cxy_wks_lstsq error,  50 ef, sum:', (Cxy_wks_lstsq - data['Cxy']).abs()[:50, :50].sum(), 'mean:', (Cxy_wks_lstsq - data['Cxy']).abs()[:50, :50].mean())
print('Cxy_wks_lstsq error, 100 ef, sum:', (Cxy_wks_lstsq - data['Cxy']).abs()[:100, :100].sum(), 'mean:', (Cxy_wks_lstsq - data['Cxy']).abs()[:100, :100].mean())
print('Cxy_wks_lstsq error, 200 ef, sum:', (Cxy_wks_lstsq - data['Cxy']).abs().sum(), 'mean:', (Cxy_wks_lstsq - data['Cxy']).abs().mean())

Cxy_lstsq error,      50 ef, sum: tensor(146.2175) mean: tensor(0.0585)
Cxy_lstsq error,     100 ef, sum: tensor(481.7635) mean: tensor(0.0482)
Cxy_lstsq error,     200 ef, sum: tensor(1554.0223) mean: tensor(0.0389)

Cxy_hks_lstsq error,  50 ef, sum: tensor(169.5795) mean: tensor(0.0678)
Cxy_hks_lstsq error, 100 ef, sum: tensor(472.1356) mean: tensor(0.0472)
Cxy_hks_lstsq error, 200 ef, sum: tensor(1363.8035) mean: tensor(0.0341)

Cxy_wks_lstsq error,  50 ef, sum: tensor(260.7804) mean: tensor(0.1043)
Cxy_wks_lstsq error, 100 ef, sum: tensor(678.8275) mean: tensor(0.0679)
Cxy_wks_lstsq error, 200 ef, sum: tensor(1659.1672) mean: tensor(0.0415)


In [189]:
######################################################
# Optimization problem with area and conformal difference
######################################################

import scipy.optimize as opt

evecs_x = data_x['evecs'].cpu().numpy()[:, :50]
evecs_y = data_y['evecs'].cpu().numpy()[:, :50]
evecs_trans_x = data_x['evecs_trans'].cpu().numpy()[:50, :]
evecs_trans_y = data_y['evecs_trans'].cpu().numpy()[:50, :]
Cxy_gt = data['Cxy'].cpu().numpy()[:50, :50]

evals_x = data_x['evals'].cpu().numpy()[:50]
evals_y = data_y['evals'].cpu().numpy()[:50]

V_gt = Cxy_gt.T @ Cxy_gt
R_gt = np.diag(-1/evals_x) @ Cxy_gt.T @ np.diag(evals_y) @ Cxy_gt

ev_sqdiff = np.square(evals_x[None, :] - evals_y[:, None])
ev_sqdiff = ev_sqdiff / ev_sqdiff.sum()


def area_difference(Cxy):
    Cxy = Cxy.reshape(Cxy_gt.shape)
    
    Cxy_T_Cxy = Cxy.T @ Cxy
    Cxy_T_Cxy = Cxy_T_Cxy / Cxy_T_Cxy.sum()
    
    return 0.5 * np.sum(np.square(
        Cxy_T_Cxy - V_gt / V_gt.sum()
        ))
    
    
def conformal_difference(Cxy):
    # return 0
    Cxy = Cxy.reshape(Cxy_gt.shape)
    
    predicted = (np.diag(-1 / evals_x[:]) @ \
        Cxy[:, :].T @ \
        np.diag(evals_y[:]) @ \
        # Cxy[:, :])[0:, 0:] - R_gt[0:, 0:]       
        Cxy[:, :])[1:, 1:]
    
    predicted = predicted / predicted.sum()
    actual = R_gt[1:, 1:] / R_gt[1:, 1:].sum()
    
    return 0.5 * np.mean(np.square(
        predicted - actual      
    ))
    

def objective_diff_operators(Cxy, info):
    
    Cxy = Cxy.reshape(Cxy_gt.shape)
    
    # descriptor preservation
    desc_pres = 0.5 * np.sum(np.square(Cxy @ evecs_trans_x - evecs_trans_y))

    # laplace commutativity    
    laplace_comm = 0.5 * (np.square(Cxy) * ev_sqdiff).sum()

    # area and conformal difference
    area_diff = area_difference(Cxy)
    conf_diff = conformal_difference(Cxy)
    
    # print every 10000 iterations
    if info['Nfeval'] % info['print_every'] == 0:
        print(f"{info['Nfeval']}: desc {desc_pres:.7f}, laplace {laplace_comm:.7f}, area {area_diff:.7f}, conf {conf_diff:.7f}")
    info['Nfeval'] += 1    
    
    return info['desc_weight']*desc_pres + laplace_comm + info['area_weight']*area_diff + info['conf_weight']*conf_diff


print('Cxy_gt')
objective_diff_operators(Cxy_gt, {'Nfeval':0, 'print_every':10000,'desc_weight': 1000, 'area_weight': 1, 'conf_weight': 1})
print('---------------------------------')


# initialization
# Cxy_0 = np.eye(Cxy_gt.shape[0]) #+ np.random.randn(*Cxy_gt.shape) * 0.05
Cxy_0 = Cxy_lstsq[:50, :50].numpy()
# Cxy_0 = np.random.randn(*Cxy_gt.shape) * 0.5


# minimize
res = opt.fmin_l_bfgs_b(
    func=objective_diff_operators,
    x0=Cxy_0.reshape(-1),
    args=({
        'Nfeval':0,
        'print_every':10000,
        'desc_weight': 1,
        'area_weight': 1,
        'conf_weight': 1
        },),
    approx_grad=True,
    maxiter=300000,
    maxfun=300000
)

Cxy_opt = res[0].reshape(Cxy_gt.shape)

Cxy_gt
0: desc 0.0139030, laplace 0.0003710, area 0.0000000, conf 0.0000000
---------------------------------
0: desc 0.0080456, laplace 0.0005256, area 0.0272858, conf 0.0000481
10000: desc 0.0082145, laplace 0.0007532, area 0.0094935, conf 0.0000046
20000: desc 0.0080711, laplace 0.0004005, area 0.0075172, conf 0.0000061
30000: desc 0.0082834, laplace 0.0003767, area 0.0053022, conf 0.0000196
40000: desc 0.0084635, laplace 0.0004066, area 0.0030422, conf 0.0000062
50000: desc 0.0085401, laplace 0.0004058, area 0.0019669, conf 0.0000058
60000: desc 0.0086851, laplace 0.0003920, area 0.0013787, conf 0.0000194
70000: desc 0.0086927, laplace 0.0003296, area 0.0004975, conf 0.0000055
80000: desc 0.0086342, laplace 0.0002922, area 0.0001527, conf 0.0000043
90000: desc 0.0085809, laplace 0.0002503, area 0.0000961, conf 0.0000037
100000: desc 0.0084872, laplace 0.0001469, area 0.0000489, conf 0.0000040
110000: desc 0.0084571, laplace 0.0000956, area 0.0000349, conf 0.0000030
120000: desc 0.0

In [179]:
ev_sqdiff = np.square(evals_x[None, :] - evals_y[:, None])
ev_sqdiff = ev_sqdiff / ev_sqdiff.sum()

def objective_laplace_comm(Cxy, info):
    
    Cxy = Cxy.reshape(Cxy_gt.shape)
    
    # descriptor preservation
    desc_pres = 0.5 * np.sum(np.square(Cxy @ evecs_trans_x - evecs_trans_y))

    # laplace commutativity
    # laplace_diff = np.sum(np.square(
    #     Cxy @ np.diag(evals_x) - np.diag(evals_y) @ Cxy
    #     ))
    
    laplace_diff = 0.5 * (np.square(Cxy) * ev_sqdiff).sum()
    
    # print every 10000 iterations
    if info['Nfeval'] % info['print_every'] == 0:
        print(f"{info['Nfeval']}: desc {desc_pres:.7f}, comm {laplace_diff:.7f}")
    info['Nfeval'] += 1    
    
    return desc_pres + laplace_diff


print('Cxy_gt')
objective_laplace_comm(Cxy_gt, {'Nfeval':0, 'print_every':10000,'desc_weight': 1000, 'area_weight': 1, 'conf_weight': 1})
print('---------------------------------')


# initialization
# Cxy_0 = np.eye(Cxy_gt.shape[0]) #+ np.random.randn(*Cxy_gt.shape) * 0.05
# Cxy_0 = Cxy_lstsq[:50, :50].numpy()
Cxy_0 = np.random.randn(*Cxy_gt.shape) * 0.5


# minimize
res_comm = opt.fmin_l_bfgs_b(
    func=objective_laplace_comm,
    x0=Cxy_0.reshape(-1),
    args=({
        'Nfeval':0,
        'print_every':5000,
        'desc_weight': 1,
        'area_weight': 1,
        'conf_weight': 1
        },),
    approx_grad=True,
    maxiter=100000,
    maxfun=100000
)
Cxy_opt_comm = res_comm[0].reshape(Cxy_gt.shape)


Cxy_gt
0: desc 0.0139030, comm 0.0003710
---------------------------------
0: desc 0.1181455, comm 0.1285492
5000: desc 0.1104173, comm 0.1142726
10000: desc 0.0207682, comm 0.0050332
15000: desc 0.0094452, comm 0.0011580
20000: desc 0.0085205, comm 0.0001767
25000: desc 0.0081553, comm 0.0001090
30000: desc 0.0081117, comm 0.0000563
35000: desc 0.0080885, comm 0.0000531
40000: desc 0.0080854, comm 0.0000512


In [128]:
objective(Cxy_opt, {'Nfeval':0, 'print_every':10000,'desc_weight': 1000, 'area_weight': 1, 'conf_weight': 1})


0: desc 0.0160948, area 33.2356864, conf 61.0985776


110.42909380248226

In [190]:
import utils.fmap_util as fmap_util
import metrics.geodist_metric as geodist_metric


def compare_with_gt(Cxy_est, Cxy_gt, evecs_x, evecs_y, verts_x, verts_y, corr_x, corr_y):
    
    # hard correspondence 
    p2p_gt = fmap_util.fmap2pointmap(
        torch.tensor(Cxy_gt),
        torch.tensor(evecs_x),
        torch.tensor(evecs_y)
        )
    p2p_est = fmap_util.fmap2pointmap(
        torch.tensor(Cxy_est, dtype=torch.float32),
        torch.tensor(evecs_x),
        torch.tensor(evecs_y)
        )
    
    # soft correspondence
    Pyx_gt = evecs_y @ Cxy_gt @ evecs_trans_x
    Pyx_est = evecs_y @ Cxy_est @ evecs_trans_x
    
    # distance matrices
    dist_x = torch.cdist(verts_x, verts_x)
    dist_y = torch.cdist(verts_y, verts_y)

    # geodesic error
    geo_err_gt = geodist_metric.calculate_geodesic_error(dist_x, corr_x, corr_y, p2p_gt, return_mean=False)    
    geo_err_est = geodist_metric.calculate_geodesic_error(dist_x, corr_x, corr_y, p2p_est, return_mean=False)
    
    # print results
    print(f'Cxy_est - Cxy_gt: {np.sum(np.abs(Cxy_est - Cxy_gt)):.2f}')
    print(f'Pyx_est - Pyx_gt: {np.sum(np.abs(Pyx_est - Pyx_gt)):.2f}')
    print(f'Geodesic error GT: {geo_err_gt.abs().sum():.2f}, EST: {geo_err_est.abs().sum():.2f}')
    
    
print('------ Cxy_opt')
compare_with_gt(
    Cxy_opt,
    Cxy_gt,
    evecs_x,
    evecs_y,
    data_x['verts'],
    data_y['verts'],
    data_x['corr'],
    data_y['corr']
    )    
    
print('------ Cxy_opt_comm')
compare_with_gt(
    Cxy_opt_comm,
    Cxy_gt,
    evecs_x,
    evecs_y,
    data_x['verts'],
    data_y['verts'],
    data_x['corr'],
    data_y['corr']
    )

print('\n------ Cxy_lstsq')
compare_with_gt(
    Cxy_lstsq[:50, :50].numpy(),
    Cxy_gt,
    evecs_x,
    evecs_y,
    data_x['verts'],
    data_y['verts'],
    data_x['corr'],
    data_y['corr']
    )

print('\n------ Cxy_hks_lstsq')
compare_with_gt(
    Cxy_hks_lstsq[:50, :50].numpy(),
    Cxy_gt,
    evecs_x,
    evecs_y,
    data_x['verts'],
    data_y['verts'],
    data_x['corr'],
    data_y['corr']
    )


------ Cxy_opt
Cxy_est - Cxy_gt: 131.31
Pyx_est - Pyx_gt: 12741.73
Geodesic error GT: 92.73, EST: 294.46
------ Cxy_opt_comm
Cxy_est - Cxy_gt: 131.79
Pyx_est - Pyx_gt: 12400.13
Geodesic error GT: 92.73, EST: 320.83

------ Cxy_lstsq
Cxy_est - Cxy_gt: 146.22
Pyx_est - Pyx_gt: 12059.97
Geodesic error GT: 92.73, EST: 387.44

------ Cxy_hks_lstsq
Cxy_est - Cxy_gt: 169.58
Pyx_est - Pyx_gt: 23494.65
Geodesic error GT: 92.73, EST: 1646.21


In [192]:
mat = np.array([
    [1, -1, -1],
    [-1, 1, -1],
    [1, 1, -1]
])
mat_inv = np.linalg.inv(mat)

mat, mat_inv, mat_inv @ mat

(array([[ 1, -1, -1],
        [-1,  1, -1],
        [ 1,  1, -1]]),
 array([[ 0. , -0.5,  0.5],
        [-0.5,  0. ,  0.5],
        [-0.5, -0.5, -0. ]]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))